In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/03 21:52:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


**Q1**: Spark Version

In [4]:
spark.version

'3.5.4'

**Q2** Question 2: Yellow October 2024

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

In [6]:
df = spark.read \
    .parquet('yellow_tripdata_2024-10.parquet')

df = df.repartition(4)

df.write.mode('overwrite').parquet('data/pq/yellow/2024/10/')

In [7]:
!ls -hl data/pq/yellow/2024/10/

total 188928
-rw-r--r--@ 1 chongwang  staff     0B Mar  3 21:52 _SUCCESS
-rw-r--r--@ 1 chongwang  staff    22M Mar  3 21:52 part-00000-82ddd449-225e-4d50-894c-267f36bb5802-c000.snappy.parquet
-rw-r--r--@ 1 chongwang  staff    22M Mar  3 21:52 part-00001-82ddd449-225e-4d50-894c-267f36bb5802-c000.snappy.parquet
-rw-r--r--@ 1 chongwang  staff    22M Mar  3 21:52 part-00002-82ddd449-225e-4d50-894c-267f36bb5802-c000.snappy.parquet
-rw-r--r--@ 1 chongwang  staff    22M Mar  3 21:52 part-00003-82ddd449-225e-4d50-894c-267f36bb5802-c000.snappy.parquet


**Q3**: How many taxi trips were there on the 15th of October?

In [8]:
from pyspark.sql import functions as F

In [9]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [10]:
df.withColumn('pickup_date', F.to_date(df.tpep_pickup_datetime)).filter("pickup_date = '2024-10-15'").count()

128893

Question 4: Longest trip

In [11]:
df.createOrReplaceTempView('trips_data')

In [12]:
spark.sql("""
SELECT
    MAX(DATE_DIFF(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime)) as longest_trip
FROM
    trips_data    
""").show()

+------------+
|longest_trip|
+------------+
|         162|
+------------+



Question 5: User Interface
Spark’s User Interface which shows the application's dashboard runs on which local port?

4040

Question 6: Least frequent pickup location zone
Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

In [13]:
df_zones = spark.read.option('Header', True).csv('taxi_zone_lookup.csv')

In [14]:
df_zones.head(5)

[Row(LocationID='1', Borough='EWR', Zone='Newark Airport', service_zone='EWR'),
 Row(LocationID='2', Borough='Queens', Zone='Jamaica Bay', service_zone='Boro Zone'),
 Row(LocationID='3', Borough='Bronx', Zone='Allerton/Pelham Gardens', service_zone='Boro Zone'),
 Row(LocationID='4', Borough='Manhattan', Zone='Alphabet City', service_zone='Yellow Zone'),
 Row(LocationID='5', Borough='Staten Island', Zone='Arden Heights', service_zone='Boro Zone')]

In [15]:
df.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [17]:
df_zones.createOrReplaceTempView('zones')

In [18]:
spark.sql("""
SELECT
    Zone
    ,COUNT(*) as n_trips
FROM
    trips_data as t
LEFT JOIN
    zones as z
ON
    t.PULocationID = z.LocationID
GROUP BY 1
ORDER BY 2 ASC

""").show()

[Stage 20:====================================>                     (5 + 3) / 8]

+--------------------+-------+
|                Zone|n_trips|
+--------------------+-------+
|Governor's Island...|      1|
|       Rikers Island|      2|
|       Arden Heights|      2|
|         Jamaica Bay|      3|
| Green-Wood Cemetery|      3|
|   Rossville/Woodrow|      4|
|       West Brighton|      4|
|       Port Richmond|      4|
|Eltingville/Annad...|      4|
|Charleston/Totten...|      4|
|         Great Kills|      6|
|        Crotona Park|      6|
|     Mariners Harbor|      7|
|Heartland Village...|      7|
|Saint George/New ...|      9|
|             Oakwood|      9|
|New Dorp/Midland ...|     10|
|       Broad Channel|     10|
|         Westerleigh|     12|
|     Pelham Bay Park|     12|
+--------------------+-------+
only showing top 20 rows

